### when prune model use the environment: 
1.ubuntu 

2.tf-nightly-gpu


* pip uninstall -y tensorflow

* pip uninstall -y tf-nightly

* pip install -U tf-nightly-gpu

* pip install tensorflow-model-optimization

In [36]:
import tensorflow as tf 
tf.enable_eager_execution()
from tensorflow_model_optimization.sparsity import keras as sparsity

### model and dataset generator

In [59]:
from Config import emotion_config as config
from utils.preprocessing import ImageToArrayPreprocessor
from utils.callbacks import EpochCheckpoint
from utils.callbacks import TrainingMonitor
from utils.io import HDF5DatasetGenerator
from utils.nn.conv import EmotionVGGNet
import os

ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator
Adam = tf.keras.optimizers.Adam
K = tf.keras.backend

trainAug = ImageDataGenerator(rotation_range=10,
                              zoom_range=0.1,
                              horizontal_flip=True,
                              rescale=1/255.0,
                              fill_mode="nearest")

valAug = ImageDataGenerator(rescale=1/255.0)
iap = ImageToArrayPreprocessor()

trainGen = HDF5DatasetGenerator(config.TRAIN_HDF5, config.BATCH_SIZE,
                               aug=trainAug, preprocessors=[iap],
                               classes=config.NUM_CLASSES)
valGen = HDF5DatasetGenerator(config.VAL_HDF5, config.BATCH_SIZE,
                              aug=valAug, preprocessors=[iap],
                              classes=config.NUM_CLASSES)


### after train and finetune, get the epoch_70.hdf5 model file, then prune the model.

In [38]:
model_path = "./ckpt/epoch_70.hdf5"
loaded_model = tf.keras.models.load_model(model_path)


### Config the prune parameters

In [39]:
import numpy as np
epochs = 3
end_step = np.ceil(1.0 * trainGen.numImages / config.BATCH_SIZE).astype(np.int32) * epochs

new_pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.20,
                                                   final_sparsity=0.50,
                                                   begin_step=0,
                                                   end_step=end_step,
                                                   frequency=100)
}

new_pruned_model = sparsity.prune_low_magnitude(loaded_model, **new_pruning_params)
new_pruned_model.summary()

opt = Adam(lr=1e-5)
new_pruned_model.compile(
    loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_conv2d ( (None, 48, 48, 32)        610       
_________________________________________________________________
prune_low_magnitude_elu (Pru (None, 48, 48, 32)        1         
_________________________________________________________________
prune_low_magnitude_batch_no (None, 48, 48, 32)        129       
_________________________________________________________________
prune_low_magnitude_conv2d_1 (None, 48, 48, 32)        18466     
_________________________________________________________________
prune_low_magnitude_elu_1 (P (None, 48, 48, 32)        1         
_________________________________________________________________
prune_low_magnitude_batch_no (None, 48, 48, 32)        129       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 24, 24, 32)       

### prune according to the config

In [40]:
logdir = "./log"

callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
]

new_pruned_model.fit_generator(trainGen.generator(),
                    steps_per_epoch=trainGen.numImages//config.BATCH_SIZE,
                    validation_data=valGen.generator(),
                    validation_steps=valGen.numImages//config.BATCH_SIZE,
                    epochs=epochs,
                    max_queue_size=config.BATCH_SIZE * 2,
                    callbacks=callbacks,
                    verbose=1)

# score = new_pruned_model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

Epoch 1/3


  1/224 [..............................] - ETA: 9:33 - loss: 0.7327 - acc: 0.7422

  2/224 [..............................] - ETA: 5:30 - loss: 0.7758 - acc: 0.7109

  3/224 [..............................] - ETA: 4:09 - loss: 0.7360 - acc: 0.7214

  4/224 [..............................] - ETA: 3:29 - loss: 0.7325 - acc: 0.7188

...

220/224 [============================>.] - ETA: 1s - loss: 0.7010 - acc: 0.7411

221/224 [============================>.] - ETA: 1s - loss: 0.7011 - acc: 0.7410

222/224 [============================>.] - ETA: 0s - loss: 0.7016 - acc: 0.7407

223/224 [============================>.] - ETA: 0s - loss: 0.7018 - acc: 0.7407

224/224 [==============================] - 90s 403ms/step - loss: 0.7012 - acc: 0.7410 - val_loss: 0.8778 - val_acc: 0.6770


In [92]:
testGen = HDF5DatasetGenerator(config.TEST_HDF5, config.BATCH_SIZE,
                              aug=valAug, preprocessors=[iap],
                              classes=config.NUM_CLASSES)


(loss, acc) = new_pruned_model.evaluate_generator(
	testGen.generator(),
	steps=testGen.numImages // config.BATCH_SIZE,
	max_queue_size=config.BATCH_SIZE * 2)

print("[INFO] accuracy: {:.2f}".format(acc * 100))

[INFO] accuracy: 66.24


### get the pruned model

In [14]:
final_model = sparsity.strip_pruning(new_pruned_model)
final_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
elu (ELU)                    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
elu_1 (ELU)                  (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)       

### save the pruned model

In [15]:
new_pruned_keras_file = "./model/pruned_model.h5"
print('Saving pruned model to: ', new_pruned_keras_file)
tf.keras.models.save_model(final_model, new_pruned_keras_file, 
                        include_optimizer=False)

Saving pruned model to:  ./model/pruned_model.h5


### convert the pruned model to tflite

In [18]:
tflite_model_file = "./model/emotion.tflite"

converter = tf.lite.TFLiteConverter.from_keras_model_file(new_pruned_keras_file)
tflite_model = converter.convert()
with open(tflite_model_file, 'wb') as f:
  f.write(tflite_model)

W0701 18:06:40.143438 139675404187392 hdf5_format.py:171] No training configuration found in save file: the model was *not* compiled. Compile it manually.


### convert the pruned model to quantized tflite

In [19]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(new_pruned_keras_file)

converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

tflite_quant_model = converter.convert()

tflite_quant_model_file = './model/emotion_quant.tflite'
with open(tflite_quant_model_file, 'wb') as f:
  f.write(tflite_quant_model)

W0701 18:08:25.330142 139675404187392 hdf5_format.py:171] No training configuration found in save file: the model was *not* compiled. Compile it manually.
